# WEB SCRAPING USING BEAUTIFULSOUP ON:
1. A simple html webpage
2. Jobs In Ghana website

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import os

with open('simplewebpage.html') as html_file:
  soup = BeautifulSoup(html_file, 'lxml')

match = soup.title.text
print(match)

Test - A Sample Website


In [ ]:
footer = soup.find('div', class_='footer').text
print(footer)


Footer Information



In [ ]:
csv_file = open('simplewebpage_scrape.csv', 'w')

csv_writer = csv.writer(csv_file)
csv_writer.writerow(['headline', 'summary'])

## writing the headers and summary of articles listed on the webpage in a csv
article = soup.find_all('div', class_ = 'article')
for art in article:
  headline = art.h2.a.text
  print(headline)
  summary = art.p.text
  print(summary)

  csv_writer.writerow([headline, summary])
csv_file.close()

Article 1 Headline
This is a summary of article 1
Article 2 Headline
This is a summary of article 2


## WEB SCRAPING ON JOBS IN GHANA WEBSITE

Here, I performed initial fetch on the website and also went through the html code to see how I would go about the project

In [ ]:

url = "https://www.jobsinghana.com/home/indexnew.php?device=d"
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'lxml')

    # Print the raw HTML to check if the category section is included
    #print(soup.prettify())
else:
    print("Failed to retrieve the page. Status code:", response.status_code)

In [ ]:
## base url for categories
catpage_url = 'https://www.jobsinghana.com/jobs/indexnew.php?device=d&cat='
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'}

## main page listing all categories
home_url = 'https://www.jobsinghana.com/home/indexnew.php?device=d'
response = requests.get(home_url, headers)

#print(response.status_code)

soup = BeautifulSoup(response.content, 'html.parser')
cat_div = soup.find('div', id = 'resorthomecats')

categories = {}

if cat_div:
  cat_links = cat_div.find_all('a')

  for item in cat_links:
    cat_name = item.text.strip()
    cat_url = item['href']

    cat_id =  cat_url.split('cat=')[-1]
    full_url = requests.compat.urljoin(home_url, cat_url)

    categories[cat_name] = cat_id
    print(f"Category: {cat_name}, URL: {cat_url}")
else:
  print("Category div not found on the page.")



200
Category: Accounting, URL: ../jobs/indexnew.php?device=d&cat=10
Category: Architecture/Construction/Real Estate, URL: ../jobs/indexnew.php?device=d&cat=20
Category: Consulting, URL: ../jobs/indexnew.php?device=d&cat=23
Category: Domestic, URL: ../jobs/indexnew.php?device=d&cat=361
Category: Facilities Management, URL: ../jobs/indexnew.php?device=d&cat=320
Category: Hospitality, URL: ../jobs/indexnew.php?device=d&cat=29
Category: International Development, URL: ../jobs/indexnew.php?device=d&cat=347
Category: Management/Administration, URL: ../jobs/indexnew.php?device=d&cat=35
Category: Mining/Petroleum, URL: ../jobs/indexnew.php?device=d&cat=40
Category: Others, URL: ../jobs/indexnew.php?device=d&cat=41
Category: Project Development, URL: ../jobs/indexnew.php?device=d&cat=84
Category: Retail / Wholesale, URL: ../jobs/indexnew.php?device=d&cat=47
Category: Social Work, URL: ../jobs/indexnew.php?device=d&cat=359
Category: Administrative, URL: ../jobs/indexnew.php?device=d&cat=16
Categ

First part of the project where the data on job categories was scraped and displayed for user

In [ ]:
url = "https://www.jobsinghana.com/home/indexnew.php?device=d"
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'lxml')

else:
    print("Failed to retrieve the page. Status code:", response.status_code)


## base url for categories
catpage_url = 'https://www.jobsinghana.com/jobs/indexnew.php?device=d&cat='
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'}

## main page listing all categories
home_url = 'https://www.jobsinghana.com/home/indexnew.php?device=d'
response = requests.get(home_url, headers)


soup = BeautifulSoup(response.content, 'html.parser')
cat_div = soup.find('div', id = 'resorthomecats')


## dictionary to store job categories and their links
categories = {}

if cat_div:
  cat_links = cat_div.find_all('a')

  for item in cat_links:
    cat_name = item.text.strip()
    cat_url = item['href']

    cat_id =  cat_url.split('cat=')[-1]
    full_url = requests.compat.urljoin(home_url, cat_url)

    categories[cat_name] = cat_id
    #print(f"Category: {cat_name}, URL: {cat_url}")
else:
  print("Category div not found on the page.")


print('Job Categories:')
for name in categories.keys():
  print(name)


Job Categories:
Accounting
Architecture/Construction/Real Estate
Consulting
Domestic
Facilities Management
Hospitality
International Development
Management/Administration
Mining/Petroleum
Others
Project Development
Retail / Wholesale
Social Work
Administrative
Automotive/Machinery/Aviation
Contract Management
Driving / Transportation
Finance
Human Resources
International Trade / Development
Manufacturing
Monitoring & Evaluation
Policy
Public Health
Sales / Business Development
Supply Chain Management
Advertising/Public Relations
Banking
Customer Service
Education
Food Service
I.T.
Legal
Marketing
NGO
Procurement
Publishing/Printing
Science
W.A.S.H.
Agriculture
Communications
Data Management
Engineering
Health and Nutrition
Insurance
Maintenance
Medical/Pharmaceutical
Operations
Programme Development
Real Estate
Security


Second and final part of the project where user enters a category to search and the various job listings and their details are returned. Job listing are also saved or appended to a csv file.

In [ ]:
import os
user_cat = input('Enter your job category: ')


# Check if category is in the dictionary
if user_cat in categories:
    cat_id = categories[user_cat]
    cat_url = catpage_url + cat_id
    print(f"\nFetching job listings for {user_cat}")

    cat_response = requests.get(cat_url, headers)
    cat_soup = BeautifulSoup(cat_response.content, 'html.parser')

    job_listings = cat_soup.find_all('div', class_='joblistitem row')

    if not job_listings:
        print("No job listings found in this category.")
    else:
        # Check if the file exists to avoid duplicating headers
        file_exists = os.path.isfile('jobsin_gh_scrape.csv')

        # Open the CSV file in append mode
        with open('jobsin_gh_scrape.csv', 'a', newline='', encoding='utf-8') as csv_file:
            csv_writer = csv.writer(csv_file)

            # Write header only if file does not already exist
            if not file_exists:
                csv_writer.writerow(['Job Title', 'URL', 'Company', 'Location', 'Base Salary', 'Employment Type', 'Expires On'])

            # Iterate through job listings and write to the CSV file
            for job in job_listings:
                title_div = job.find('h3')
                if title_div:
                    title_anchor = title_div.find('a')
                    job_title = title_anchor.text.strip() if title_anchor else "N/A"
                    job_url = requests.compat.urljoin(cat_url, title_anchor['href']) if title_anchor else "N/A"

                    company_tag = job.find('h4')
                    company_name = company_tag.text.strip() if company_tag else "N/A"

                    location_tag = job.find('span', property='addressLocality')
                    location = location_tag.text.strip() if location_tag else "N/A"

                    salary_tag = job.find('span', property='baseSalary')
                    base_salary = salary_tag.text.strip() if salary_tag else "N/A"

                    employment_type_tag = job.find('span', property='employmentType')
                    employment_type = employment_type_tag.text.strip() if employment_type_tag else "N/A"

                    expiry_tag = job.find('span', class_='posted_date')
                    expiry_date = expiry_tag.text.strip().replace("Expires", "").strip() if expiry_tag else "N/A"

                    print(f"Title: {job_title}")
                    print(f"URL: {job_url}")
                    print(f"Company: {company_name}")
                    print(f"Location: {location}")
                    print(f"Base Salary: {base_salary}")
                    print(f"Employment Type: {employment_type}")
                    print(f"Expiry Date: {expiry_date}")
                    print("-" * 50)
                    csv_writer.writerow([job_title, job_url, company_name, location, base_salary, employment_type, expiry_date])

else:
    print("Invalid category. Enter a valid category.")

Enter your job category: I.T.

Fetching job listings for I.T.
Title: Data Annotators
URL: https://www.jobsinghana.com/jobs/indexnew.php?&device=d&view=52342&cat=31
Company: Aya Data
Location: N/A
Base Salary: N/A
Employment Type: Full-time
Expiry Date: Nov 27, 2024
--------------------------------------------------
Title: Primary IT Teacher
URL: https://www.jobsinghana.com/jobs/indexnew.php?&device=d&view=52299&cat=31
Company: GIU International Christian Academy
Location: N/A
Base Salary: Competitive
Employment Type: Full-time
Expiry Date: Nov 20, 2024
--------------------------------------------------
Title: IT Officer
URL: https://www.jobsinghana.com/jobs/indexnew.php?&device=d&view=52297&cat=31
Company: Confidential
Location: N/A
Base Salary: N/A
Employment Type: Full-time
Expiry Date: Nov 20, 2024
--------------------------------------------------
Title: Junior Software Developer
URL: https://www.jobsinghana.com/jobs/indexnew.php?&device=d&view=52261&cat=31
Company: Amplio Ghana
Lo

Optimized code with functions

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import os

CATPAGE_URL = 'https://www.jobsinghana.com/jobs/indexnew.php?device=d&cat='
HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'}
HOME_URL = 'https://www.jobsinghana.com/home/indexnew.php?device=d'

## function to fetch job categories and take user input on category to search for
def scrape():
    response = requests.get(HOME_URL, headers=HEADERS)
    soup = BeautifulSoup(response.content, 'html.parser')
    cat_div = soup.find('div', id='resorthomecats')
    categories = {}

    ## Extract categories if available
    if cat_div:
        cat_links = cat_div.find_all('a')
        for item in cat_links:
            cat_name = item.text.strip()
            cat_id = item['href'].split('cat=')[-1]
            categories[cat_name] = cat_id
    else:
        print("Category div not found on the page.")
        return None

    print('JOB CATEGORIES:')
    for name in categories.keys():
        print(name)

    user_cat = input('Enter your job category: ')
    if user_cat in categories:
        return categories[user_cat]
    else:
        print("Invalid category. Exiting.")
        return None

## function to parse listing from the category searched and return each job details
def parse(category_id):
    cat_url = CATPAGE_URL + category_id
    cat_response = requests.get(cat_url, headers=HEADERS)
    cat_soup = BeautifulSoup(cat_response.content, 'html.parser')
    job_listings = cat_soup.find_all('div', class_='joblistitem row')

    jobs_data = []
    for job in job_listings:
        title_div = job.find('h3')
        title_anchor = title_div.find('a') if title_div else None
        job_title = title_anchor.text.strip() if title_anchor else "N/A"
        job_url = requests.compat.urljoin(cat_url, title_anchor['href']) if title_anchor else "N/A"
        company_name = job.find('h4').text.strip() if job.find('h4') else "N/A"
        location = job.find('span', property='addressLocality').text.strip() if job.find('span', property='addressLocality') else "N/A"
        base_salary = job.find('span', property='baseSalary').text.strip() if job.find('span', property='baseSalary') else "N/A"
        employment_type = job.find('span', property='employmentType').text.strip() if job.find('span', property='employmentType') else "N/A"
        expiry_date = job.find('span', class_='posted_date').text.strip().replace("Expires", "").strip() if job.find('span', class_='posted_date') else "N/A"

        jobs_data.append({
            'Job Title': job_title,
            'URL': job_url,
            'Company': company_name,
            'Location': location,
            'Base Salary': base_salary,
            'Employment Type': employment_type,
            'Expires On': expiry_date
        })

    return jobs_data

## function to return job details and append them to a csv file or create a new one if file doesn't exist
def output(jobs_data):
    if not jobs_data:
        print("No job listings found.")
        return

    for job in jobs_data:
        print("\n".join([f"{key}: {value}" for key, value in job.items()]))
        print("-" * 50)

    ## Define file name and check if file already exists
    file_name = 'jobsin_gh_scrape.csv'
    file_exists = os.path.isfile(file_name)

    with open(file_name, 'a' if file_exists else 'w', newline='', encoding='utf-8') as csv_file:
        csv_writer = csv.writer(csv_file)
        ## Write headers only if file does not already exist
        if not file_exists:
            csv_writer.writerow(['Job Title', 'URL', 'Company', 'Location', 'Base Salary', 'Employment Type', 'Expires On'])
        for job in jobs_data:
            csv_writer.writerow(job.values())

    print(f"Job listings have been {'appended to' if file_exists else 'saved to'} {file_name}.")

def main():
    category_id = scrape()
    if category_id:
        jobs_data = parse(category_id)
        output(jobs_data)

# Run the main function
if __name__ == "__main__":
    main()

JOB CATEGORIES:
Accounting
Architecture/Construction/Real Estate
Consulting
Domestic
Facilities Management
Hospitality
International Development
Management/Administration
Mining/Petroleum
Others
Project Development
Retail / Wholesale
Social Work
Administrative
Automotive/Machinery/Aviation
Contract Management
Driving / Transportation
Finance
Human Resources
International Trade / Development
Manufacturing
Monitoring & Evaluation
Policy
Public Health
Sales / Business Development
Supply Chain Management
Advertising/Public Relations
Banking
Customer Service
Education
Food Service
I.T.
Legal
Marketing
NGO
Procurement
Publishing/Printing
Science
W.A.S.H.
Agriculture
Communications
Data Management
Engineering
Health and Nutrition
Insurance
Maintenance
Medical/Pharmaceutical
Operations
Programme Development
Real Estate
Security
Enter your job category: NGO
Job Title: Marine And Coastal Projects Manager
URL: https://www.jobsinghana.com/jobs/indexnew.php?&device=d&view=52296&cat=372
Company: Hen

Challenges faced: The salary column didn't have a consistent format so it will need further cleaning after it has been extracted.